## Business Understanding 
Purpose: Ask relevant questions and define objectives for the problem that needs to be tackled

## Background
In recent years, the range of funding options for projects created by individuals and small companies has expanded considerably. In addition to savings, bank loans, friends & family funding and other traditional options, crowdfunding has become a popular and readily available alternative.

Kickstarter, founded in 2009, is one particularly well-known and popular crowdfunding platform. It has an all-or-nothing funding model, whereby a project is only funded if it meets its goal amount; otherwise no money is given by backers to a project. A huge variety of factors contribute to the success or failure of a project — in general, and also on Kickstarter. Some of these are able to be quantified or categorized, which allows for the construction of a model to attempt to predict whether a project will succeed or not. The aim of this project is to construct such a model and also to analyse Kickstarter project data more generally, in order to help potential project creators assess whether or not Kickstarter is a good funding option for them, and what their chances of success are.

### Final Deliverables


* Well designed presentation for non-technical stakeholders outlining findings and recommendations, as well as future work (10min presentation).
* Jupyter notebook following Data Science Lifecycle

### Things to think about

* Try different (at least 3) machine learning algorithms to check which performs best on the problem at hand
* What would be right performance metric: Precision, recall, accuracy, F1 score, or something else? (Check TPR?)
* Check for data imbalance


## Key Question 

We currently hold a task by Kickstarter to come up with a model to predict in a first step whether is project is likely to be successful, given certain project parameters. In a second step (out of scope), Kickstarter would like to be able to provide a good goal recommendation for creators( for example using staff picks etc.)

* Given certain project parameters, __is a campaign likely to succeed or fail?__ --> classification
* what would e a __reasonable goal reccomendation for creators__ --> regression



## Feature Glossary

Features included in model

* Target : state
*
*
*

## Dataset Description

- **backers_count**: Amount of people who backed this project
- **category**: 
- **country**: Country the project owner lives in
- **created_at**: Date when the prjoect was created
- **currency**: Currency of the country where the owner lives in
- **currency_trailing_code**: 
- **current_currency**: 
- **deadline**: Date until the project can be backed
- **disable_communication**: If the communication with owner was disabled or not
- **fx_rate**: Foreign exchange rate
- **goal**: Project is only funded when the goal amount is reached
- **launched_at**: Date when the project was launced
- **spotlight**: Highlighted projects (available to all projects that are successfully funded)
- **staff_pick**: Promissing project picked by Kickstarter employees
- **state**: Project status
- **state_changed_at**: Date when state changed the last time
- **static_usd_rate**: static USD Convergen rate at time
- **usd_pledged**: pledge amount converted to USD using Static_usd_rate


## Dataset New/Added Feature Description

- **campaign_days**: Days the Project was live
- **pledged_over**: Amount Pledged surpassing the Goal(with converted pledge amount) 
- **pre_launched_days**: Days before the Project was launched


## Target Metric
* F1, Accuracy, recall, which one? 

## Outcome / Reccomendations
*
*
*

## Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.gridspec import GridSpec
import scipy as sc
from scipy.stats import kstest
import seaborn as sns
import math

#Data mining
import os, glob


## Dashboard
Purpose : Define global variables and visuals

In [3]:
random_state = 100
test_size = 0.3
sns.set(style = "white")

## Data Mining

In [ ]:
# Import multiple Kickstarter csv files and merge into one dataframe

path = "data-2"
all_files = glob.glob(os.path.join(path, "*.csv"))

all_df = []
for f in all_files:
    df = pd.read_csv(f, sep=',')
    df['file'] = f.split('/')[-1]
    all_df.append(df)
    
merged_df = pd.concat(all_df, ignore_index=True, sort=True)

In [5]:
merged_df = pd.read_csv('data-2/Kickstarter_all.csv') ### brauche ich wenn ich den Anderen Kram nicht laufen lassen will

## Inspection and Data Cleaning

In [ ]:
merged_df.info()

In [ ]:
#save the merged data as .zip
compression_opts = dict(method='zip', archive_name='out.csv')  
merged_df.to_csv('out.zip', index=False, compression=compression_opts)

In [ ]:
# Display shape of "data"
merged_df.shape

In [ ]:
merged_df.head()

In [ ]:
merged_df.columns

In [ ]:
merged_df.groupby('state').count()

In [ ]:
pd.isnull(merged_df).sum()

## Data Handling

In [81]:
# create a dataset for Inspection
final = merged_df.copy()

### Dropping Data

In [29]:
drop_list = []

#### Dropping features with missing values

In [30]:
drop_missing_values = ['blurb', 'friends', 'is_backing', 'is_starred', 'permissions', 'usd_type', 'location']
drop_list.extend(drop_missing_values)
final = final.drop(drop_missing_values, axis = 1)


#### Dropping useless features 

In [31]:
drop_useless_features = ['creator', 'currency_symbol', 'name', 'photo', 'profile', 'slug', 'source_url', 'urls', 'file']
drop_list.extend(drop_useless_features)
final = final.drop(drop_useless_features, axis = 1)

#### Dropping redundant features

In [32]:
drop_redundant_features = ['pledged', 'usd_pledged']
drop_list.extend(drop_redundant_features)
final = final.drop(drop_redundant_features, axis = 1)

In [63]:
drop_list

['blurb',
 'friends',
 'is_backing',
 'is_starred',
 'permissions',
 'usd_type',
 'location',
 'creator',
 'currency_symbol',
 'name',
 'photo',
 'profile',
 'slug',
 'source_url',
 'urls',
 'file',
 'pledged',
 'usd_pledged']

#### Replacing features

In [82]:
def clean_category(DataFrame): 
    cat_list = []
    subcat_list = []
    for e in DataFrame.category:
        string_list = e.split(',')
        if '/' in string_list[2]:
            cat_list.append(string_list[2].split('/')[0][8:])
            subcat_list.append(string_list[2].split('/')[1][:-1])
        else:
            cat_list.append(string_list[2][8:-1])
            subcat_list.append('None')
    DataFrame['category'] = cat_list
    DataFrame['sub_category'] = subcat_list
    return DataFrame

In [85]:
modified_list = ['category']

In [83]:
final = clean_category(final)

In [84]:
final.category.unique()

array(['fashion', 'games', 'music', 'publishing', 'theater', 'food',
       'art', 'photography', 'technology', 'dance', 'design',
       'film & video', 'crafts', 'comics', 'journalism'], dtype=object)

### Time conversions



In [87]:
modified_list.extend(['launched_at', 'deadline', 'created_at', 'state_changed_at'])         

['category', 'launched_at', 'deadline', 'created_at', 'state_changed_at']

In [ ]:
#converting unix time 
final.launched_at = pd.to_datetime(final.launched_at,unit='s',infer_datetime_format=True)
final.deadline = pd.to_datetime(final.deadline,unit='s',infer_datetime_format=True)
final.created_at = pd.to_datetime(final.created_at,unit='s',infer_datetime_format=True)
final.state_changed_at = pd.to_datetime(final.state_changed_at,unit='s',infer_datetime_format=True)

### Writing df changes

In [92]:
feature_list = list(merged_df.columns)

df_features = pd.DataFrame(feature_list,columns =['features'])
df_features['dropped'] = df_features.features.isin(drop_list)
df_features['drop_reason'] = ['missing_values' if x in drop_missing_values \
                              else 'useless' if x in drop_useless_features \
                              else 'redundant' if x in drop_redundant_features \
                              else 'None' for x in df_features['features']]
df_features['modified'] = df_features.features.isin(modified_list)

In [93]:
df_features

,features,dropped,drop_reason,modified
0,backers_count,False,None,False
1,blurb,True,missing_values,False
2,category,False,None,True
3,converted_pledged_amount,False,None,False
4,country,False,None,False
5,created_at,False,None,True
6,creator,True,useless,False
7,currency,False,None,False
8,currency_symbol,True,useless,False
9,currency_trailing_code,False,None,False


# Data Exploration
Purpose: we gotta form a hypotheses / story about our defined problem by visually analyzing the data

In [ ]:
# Seperate continious vs. categorical variables
#data_cat_col = ['category','country','...']
#data_cant_col = [x for x in final if x not in data_cat_col]
#data_cat = final[data_cat_col]
#data_cotnt = final[data_cont_col]

In [ ]:
# Check if scaling is needed ( we can do this by looking at the .skew()
#final.skew()

In [ ]:
#Plot correlation heatmap for continious values



In [ ]:
#Plot a histogram of our Target 'state' and see if it needs scaling for future work

In [ ]:
# Plot other things... :)

# Feature Engineering

In [ ]:
#create new features
final['pledged_over'] = final.converted_pledged_amount - final.goal 
final['campaign_days'] = final.deadline - final.launched_at 
final['pre_launched_days'] = final.launched_at - final.created_at

# Preprocessing (Train/Test Split and Basemodel)
In order to apply modelling on different dataset types, we should consider a nice way to do the splits.



In [ ]:
#define predictors and target variable X,y

In [ ]:
#Split data into training and testing sets

In [ ]:
# create a dummy classifier model as Basemodel
#dum_clf = DummyClassifier(strategy='most_frequent').fit(X_train,y_train)
#y_pred_dum_clf = dum_clf.predict(X_test)

In [ ]:
#use standard scaler on X_train and y_train

# Predictive Modelling
Purpose: Train machine learning models (supervised), evaluate their performance and use them to make predictions

In [ ]:
#logistic regression

In [ ]:
#Random Forest Classifier

In [ ]:
#Support Vector Machines (use classifier)

In [ ]:
#maybe AdaBoost

# Ensemble Methods

In [ ]:
#use KNN,SVC,DTC,Randomforestclassifier,XGB....

# Future Work

In [ ]:
#use maybe RandomizedSearchCV on RandomForest or any given Algorithm

# Data Visualisation
Purpose: Communicate the findings with stakeholders using plots and interactive visualisations

# Findings 
Purpose: Summarize the key results and findings

# Future Work

In [ ]:
# To do: save final df as csv
#compression_opts = dict(method='zip', archive_name='Kickstarter_all_clean.csv')  
#final.to_csv('Kickstarter_all_clean.zip', index=False, compression=compression_opts)